<a href="https://colab.research.google.com/github/NirTaube/Simple-Projects/blob/main/TrendScape%2Bai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pytrends

In [ ]:
from pytrends.request import TrendReq
import matplotlib.pyplot as plt
import pandas as pd


In [ ]:
pytrends = TrendReq(hl='en-US', tz=360)


In [ ]:
num_keywords = min(25, int(input("Enter the number of keywords (up to 25): ")))
keywords = [input(f"Enter keyword {i+1}: ") for i in range(num_keywords)]


Enter the number of keywords (up to 25): 5
Enter keyword 1: Finance SAAS
Enter keyword 2: Sales SAAS
Enter keyword 3: AI SAAS
Enter keyword 4: CRM SAAS
Enter keyword 5: ERP SAAS


In [ ]:
pytrends.build_payload(keywords, timeframe='today 5-y', geo='', gprop='')
interest_over_time = pytrends.interest_over_time()


In [ ]:
import plotly.graph_objects as go

def plot_interest_over_time(data, keys):
    fig = go.Figure()

    for key in keys:
        if key in data.columns:
            fig.add_trace(go.Scatter(x=data.index, y=data[key], mode='lines', name=key, hoverinfo='x+y+name'))

    fig.update_layout(
        title='Google Trends Over Time',
        xaxis_title='Date',
        yaxis_title='Interest',
        xaxis=dict(showline=True, showgrid=True, showticklabels=True, linecolor='rgb(204, 204, 204)', linewidth=2, ticks='outside', tickfont=dict(family='Arial', size=12, color='rgb(82, 82, 82)')),
        yaxis=dict(showgrid=True, zeroline=False, showline=False, showticklabels=True),
        autosize=True,
        margin=dict(autoexpand=True, l=100, r=20, t=110),
        showlegend=True
    )

    fig.show()

plot_interest_over_time(interest_over_time, keywords)


In [ ]:
# Ensure only numeric columns are considered
numeric_data = interest_over_time.drop(columns=['isPartial'], errors='ignore')  # Dropping the 'isPartial' column if present
highest_interest = numeric_data.max()
highest_interest_keywords = highest_interest.idxmax() if not highest_interest.empty else "No data available"

time_intervals = ['1w', '1m', '3m', '6m', '12m']
percent_changes = {}

for interval in time_intervals:
    interval_data = interest_over_time.last(interval).iloc[0]
    # Calculate percentage change for each keyword
    percent_change = (interval_data - numeric_data.iloc[0]) / numeric_data.iloc[0] * 100
    percent_changes[interval] = percent_change

# Convert the dictionary to a DataFrame
percent_change_df = pd.DataFrame(percent_changes)
percent_change_df.head()


,1w,1m,3m,6m,12m
AI SAAS,683.333333,650.0,683.333333,416.666667,83.333333
CRM SAAS,90.0,110.0,110.0,90.0,150.0
ERP SAAS,88.888889,66.666667,177.777778,44.444444,88.888889
Finance SAAS,14.285714,85.714286,85.714286,14.285714,42.857143
Sales SAAS,54.166667,77.083333,91.666667,64.583333,87.5


In [ ]:
import plotly.express as px

# Drop the 'isPartial' column from numeric_data
numeric_data = interest_over_time.drop(columns=['isPartial'], errors='ignore')

# Drop the 'isPartial' column from percent_change_df
percent_change_df = percent_change_df.dropna()
# Create a heatmap using Plotly Express
fig = px.imshow(percent_change_df,
                x=percent_change_df.columns,
                y=percent_change_df.index,
                color_continuous_scale='reds',
                title='Percentage Increase in Interest Over Time Intervals - Heatmap')

# Update the layout
fig.update_layout(
    xaxis=dict(side='top'),
    xaxis_title='Keywords',
    yaxis_title='Time Intervals',
    autosize=True,
    margin=dict(autoexpand=True, l=100, r=20, t=110),
    showlegend=True
)

# Show the plot
fig.show()


In [ ]:
import requests

with open("api.txt", "r") as file:
    api_key = file.read()

def get_related_terms_from_chatgpt(keyword, num_results=5):
    # Define the endpoint, headers, and payload for the API call
    endpoint = "https://api.openai.com/v1/engines/text-davinci-003/completions"
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    data = {
        "prompt": f"Suggest related terms for '{keyword}':",
        "max_tokens": 50  # You can adjust this based on your needs
    }

    # Make the API call
    response = requests.post(endpoint, headers=headers, json=data)
    response_json = response.json()

    # Extract the model's response and parse it to get a list of terms
    model_output = response_json.get("choices", [{}])[0].get("text", "").strip()
    related_terms = model_output.split(",")[:num_results]  # Limit to num_results terms

    return [term.strip() for term in related_terms]

# Call the function with user input and get related terms (limiting to 5 results)
related_terms = get_related_terms_from_chatgpt(keywords, num_results=5)

# Print the related terms nicely
if related_terms:
    print("Related Terms:")
    for term in related_terms:
        print("- " + term)
else:
    print("No related terms found.")


Related Terms:
- Financial Reporting SAAS
- Accounting SAAS
- Lead Management SAAS
- Customer Support SAAS
- Project Management SAAS


In [ ]:
pytrends.build_payload(related_terms, timeframe='today 5-y', geo='', gprop='')
interest_over_time= pytrends.interest_over_time()

In [ ]:
import plotly.graph_objects as go

def plot_interest_over_time(data, keys):
    fig = go.Figure()

    for key in keys:
        if key in data.columns:
            fig.add_trace(go.Scatter(x=data.index, y=data[key], mode='lines', name=key, hoverinfo='x+y+name'))

    fig.update_layout(
        title='Google Trends Over Time',
        xaxis_title='Date',
        yaxis_title='Interest',
        xaxis=dict(showline=True, showgrid=True, showticklabels=True, linecolor='rgb(204, 204, 204)', linewidth=2, ticks='outside', tickfont=dict(family='Arial', size=12, color='rgb(82, 82, 82)')),
        yaxis=dict(showgrid=True, zeroline=False, showline=False, showticklabels=True),
        autosize=True,
        margin=dict(autoexpand=True, l=100, r=20, t=110),
        showlegend=True
    )

    fig.show()

plot_interest_over_time(interest_over_time, related_terms)

In [ ]:
# Ensure only numeric columns are considered
numeric_data = interest_over_time.drop(columns=['isPartial'], errors='ignore')  # Dropping the 'isPartial' column if present
highest_interest = numeric_data.max()
highest_interest_keywords = highest_interest.idxmax() if not highest_interest.empty else "No data available"

time_intervals = ['1w', '1m', '3m', '6m', '12m']
percent_changes = {}

for interval in time_intervals:
    interval_data = interest_over_time.last(interval).iloc[0]
    # Calculate percentage change for each keyword
    percent_change = (interval_data - numeric_data.iloc[0]) / numeric_data.iloc[0] * 100
    percent_changes[interval] = percent_change

# Convert the dictionary to a DataFrame
percent_change_df = pd.DataFrame(percent_changes)
percent_change_df.head()


,1w,1m,3m,6m,12m
Accounting SAAS,-58.181818,-30.909091,23.636364,-14.545455,-38.181818
Customer Support SAAS,-100.0,-100.0,45.833333,-100.0,-37.5
Financial Reporting SAAS,NaN,NaN,NaN,NaN,NaN
Lead Management SAAS,NaN,NaN,NaN,NaN,NaN
Project Management SAAS,-100.0,-37.5,25.0,43.75,6.25


In [ ]:
print(highest_interest)

Financial Reporting SAAS      0
Accounting SAAS             100
Lead Management SAAS          0
Customer Support SAAS        76
Project Management SAAS      73
dtype: int64


In [ ]:
import plotly.express as px

# Drop the 'isPartial' column from numeric_data
numeric_data = interest_over_time.drop(columns=['isPartial'], errors='ignore')

# Drop the 'isPartial' column from percent_change_df
percent_change_df = percent_change_df.dropna()
# Create a heatmap using Plotly Express
fig = px.imshow(percent_change_df,
                x=percent_change_df.columns,
                y=percent_change_df.index,
                color_continuous_scale='reds',
                title='Percentage Increase in Interest Over Time Intervals - Heatmap')

# Update the layout
fig.update_layout(
    xaxis=dict(side='top'),
    xaxis_title='Keywords',
    yaxis_title='Time Intervals',
    autosize=True,
    margin=dict(autoexpand=True, l=100, r=20, t=110),
    showlegend=True
)

# Show the plot
fig.show()
